# Hotel Search Agent Tutorial - Priority 1 Implementation

This notebook demonstrates the Agent Catalog hotel search agent using LangChain with Couchbase vector store and Arize Phoenix evaluation. Uses Priority 1 AI services with standard OpenAI wrappers and Capella (simple & fast).


## Setup and Imports

Import all necessary modules for the hotel search agent using self-contained setup.


In [1]:
import base64
import getpass
import httpx
import json
import logging
import os
import sys
import time
from datetime import timedelta
from typing import Any, Dict, List, Optional, Tuple

import agentc
import dotenv
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.management.buckets import BucketType, CreateBucketSettings
from couchbase.management.search import SearchIndex
from couchbase.options import ClusterOptions
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_couchbase.vectorstores import CouchbaseVectorStore
from tqdm import tqdm

# Setup logging to output to stdout for Colab environments
root_logger = logging.getLogger()
if not root_logger.handlers:
    handler = logging.StreamHandler(sys.stdout)
    formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
    handler.setFormatter(formatter)
    root_logger.addHandler(handler)
root_logger.setLevel(logging.INFO)

# Setup logging for this module
logger = logging.getLogger(__name__)

# Reduce noise from various libraries during embedding/vector operations
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)

# Load environment variables
dotenv.load_dotenv(override=True)

# Set default values for travel-sample bucket configuration
DEFAULT_BUCKET = "travel-sample"
DEFAULT_SCOPE = "agentc_data"
DEFAULT_COLLECTION = "hotel_data"
DEFAULT_INDEX = "hotel_data_index"
DEFAULT_CAPELLA_API_EMBEDDING_MODEL = "Snowflake/snowflake-arctic-embed-l-v2.0"
DEFAULT_CAPELLA_API_LLM_MODEL = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"


## Self-Contained Setup Functions

Define all necessary setup functions inline for a self-contained notebook.


In [2]:
def setup_environment():
    """Setup default environment variables for agent operations."""
    defaults = {
        "CB_BUCKET": "travel-sample",
        "CB_SCOPE": "agentc_data",
        "CB_COLLECTION": "hotel_data",
        "CB_INDEX": "hotel_data_index",
        "CAPELLA_API_EMBEDDING_MODEL": "Snowflake/snowflake-arctic-embed-l-v2.0",
        "CAPELLA_API_LLM_MODEL": "deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
    }
    
    for key, value in defaults.items():
        if not os.getenv(key):
            os.environ[key] = value
    
    logger.info("✅ Environment variables configured")


def test_capella_connectivity(api_key: str = None, endpoint: str = None) -> bool:
    """Test connectivity to Capella AI services."""
    try:
        test_key = api_key or os.getenv("CAPELLA_API_EMBEDDINGS_KEY") or os.getenv("CAPELLA_API_LLM_KEY")
        test_endpoint = endpoint or os.getenv("CAPELLA_API_ENDPOINT")
        
        if not test_key or not test_endpoint:
            return False
        
        # Simple connectivity test
        headers = {"Authorization": f"Bearer {test_key}"}
        
        with httpx.Client(timeout=10.0) as client:
            response = client.get(f"{test_endpoint.rstrip('/')}/v1/models", headers=headers)
            return response.status_code < 500
    except Exception as e:
        logger.warning(f"⚠️ Capella connectivity test failed: {e}")
        return False


def setup_ai_services(framework: str = "langchain", temperature: float = 0.0, application_span=None):
    """Priority 1: Capella AI with OpenAI wrappers (simple & fast) for LangChain."""
    embeddings = None
    llm = None
    
    logger.info(f"🔧 Setting up Priority 1 AI services for {framework} framework...")
    
    # Priority 1: Capella AI with direct API keys and OpenAI wrappers
    if not embeddings and os.getenv("CAPELLA_API_ENDPOINT") and os.getenv("CAPELLA_API_EMBEDDINGS_KEY"):
        try:
            endpoint = os.getenv("CAPELLA_API_ENDPOINT")
            api_key = os.getenv("CAPELLA_API_EMBEDDINGS_KEY")
            model = os.getenv("CAPELLA_API_EMBEDDING_MODEL")
            
            # Handle endpoint that may or may not already have /v1 suffix
            if endpoint.endswith('/v1'):
                api_base = endpoint
            else:
                api_base = f"{endpoint}/v1"
            
            # Debug logging - same pattern as working test
            logger.info(f"🔧 Endpoint: {endpoint}")
            logger.info(f"🔧 Model: {model}")
            logger.info(f"🔧 API Base: {api_base}")
            
            embeddings = OpenAIEmbeddings(
                model=model,
                api_key=api_key,
                base_url=api_base,
                check_embedding_ctx_length=False,  # Fix for asymmetric models
            )
            logger.info("✅ Using Priority 1: Capella AI embeddings (OpenAI wrapper)")
        except Exception as e:
            logger.error(f"❌ Priority 1 Capella AI embeddings failed: {type(e).__name__}: {e}")
    
    if not llm and os.getenv("CAPELLA_API_ENDPOINT") and os.getenv("CAPELLA_API_LLM_KEY"):
        try:
            endpoint = os.getenv("CAPELLA_API_ENDPOINT")
            llm_key = os.getenv("CAPELLA_API_LLM_KEY")
            llm_model = os.getenv("CAPELLA_API_LLM_MODEL")
            
            # Handle endpoint that may or may not already have /v1 suffix
            if endpoint.endswith('/v1'):
                api_base = endpoint
            else:
                api_base = f"{endpoint}/v1"
            
            # Debug logging
            logger.info(f"🔧 LLM Endpoint: {endpoint}")
            logger.info(f"🔧 LLM Model: {llm_model}")
            logger.info(f"🔧 LLM API Base: {api_base}")
            
            llm = ChatOpenAI(
                model=llm_model,
                base_url=api_base,
                api_key=llm_key,
                temperature=temperature,
            )
            # Test the LLM works
            test_response = llm.invoke("Hello")
            logger.info("✅ Using Priority 1: Capella AI LLM (OpenAI wrapper)")
        except Exception as e:
            logger.error(f"❌ Priority 1 Capella AI LLM failed: {type(e).__name__}: {e}")
            llm = None
    
    # Fallback: OpenAI
    if not embeddings and os.getenv("OPENAI_API_KEY"):
        try:
            embeddings = OpenAIEmbeddings(
                model="text-embedding-3-small",
                api_key=os.getenv("OPENAI_API_KEY"),
            )
            logger.info("✅ Using OpenAI embeddings fallback")
        except Exception as e:
            logger.warning(f"⚠️ OpenAI embeddings failed: {e}")
    
    if not llm and os.getenv("OPENAI_API_KEY"):
        try:
            llm = ChatOpenAI(
                model="gpt-4o",
                api_key=os.getenv("OPENAI_API_KEY"),
                temperature=temperature,
            )
            logger.info("✅ Using OpenAI LLM fallback")
        except Exception as e:
            logger.warning(f"⚠️ OpenAI LLM failed: {e}")
    
    if not embeddings:
        raise ValueError("❌ No embeddings service could be initialized")
    if not llm:
        raise ValueError("❌ No LLM service could be initialized")
    
    logger.info(f"✅ Priority 1 AI services setup completed for {framework}")
    return embeddings, llm


# Setup environment
setup_environment()

# Test Capella AI connectivity if configured
if os.getenv("CAPELLA_API_ENDPOINT"):
    if not test_capella_connectivity():
        logger.warning("❌ Capella AI connectivity test failed. Will use fallback models.")
else:
    logger.info("ℹ️ Capella API not configured - will use fallback models")


2025-09-11 13:31:49,189 - __main__ - INFO - ✅ Environment variables configured


## CouchbaseClient Class

Define the CouchbaseClient for all database operations and LangChain agent creation.


In [3]:
class CouchbaseClient:
    """Centralized Couchbase client for all database operations."""

    def __init__(self, conn_string: str, username: str, password: str, bucket_name: str):
        """Initialize Couchbase client with connection details."""
        self.conn_string = conn_string
        self.username = username
        self.password = password
        self.bucket_name = bucket_name
        self.cluster = None
        self.bucket = None
        self._collections = {}

    def connect(self):
        """Establish connection to Couchbase cluster."""
        try:
            auth = PasswordAuthenticator(self.username, self.password)
            options = ClusterOptions(auth)

            # Use WAN profile for better timeout handling with remote clusters
            options.apply_profile("wan_development")
            self.cluster = Cluster(self.conn_string, options)
            self.cluster.wait_until_ready(timedelta(seconds=20))
            logger.info("Successfully connected to Couchbase")
            return self.cluster
        except Exception as e:
            raise ConnectionError(f"Failed to connect to Couchbase: {e!s}")

    def setup_collection(self, scope_name: str, collection_name: str, clear_existing_data: bool = False):
        """Setup collection - create scope and collection if they don't exist."""
        try:
            # Ensure cluster connection
            if not self.cluster:
                self.connect()

            # For travel-sample bucket, assume it exists
            if not self.bucket:
                self.bucket = self.cluster.bucket(self.bucket_name)
                logger.info(f"Connected to bucket '{self.bucket_name}'")

            # Setup scope
            bucket_manager = self.bucket.collections()
            scopes = bucket_manager.get_all_scopes()
            scope_exists = any(scope.name == scope_name for scope in scopes)

            if not scope_exists and scope_name != "_default":
                logger.info(f"Creating scope '{scope_name}'...")
                bucket_manager.create_scope(scope_name)
                logger.info(f"Scope '{scope_name}' created successfully")

            # Setup collection - clear if exists, create if doesn't
            collections = bucket_manager.get_all_scopes()
            collection_exists = any(
                scope.name == scope_name
                and collection_name in [col.name for col in scope.collections]
                for scope in collections
            )

            if collection_exists:
                if clear_existing_data:
                    logger.info(f"Collection '{collection_name}' exists, clearing data...")
                    self.clear_collection_data(scope_name, collection_name)
                else:
                    logger.info(f"Collection '{collection_name}' exists, keeping existing data...")
            else:
                logger.info(f"Creating collection '{collection_name}'...")
                bucket_manager.create_collection(scope_name, collection_name)
                logger.info(f"Collection '{collection_name}' created successfully")

            time.sleep(3)

            # Create primary index
            try:
                self.cluster.query(
                    f"CREATE PRIMARY INDEX IF NOT EXISTS ON `{self.bucket_name}`.`{scope_name}`.`{collection_name}`"
                ).execute()
                logger.info("Primary index created successfully")
            except Exception as e:
                logger.warning(f"Error creating primary index: {e}")

            logger.info("Collection setup complete")
            return self.bucket.scope(scope_name).collection(collection_name)

        except Exception as e:
            raise RuntimeError(f"Error setting up collection: {e!s}")

    def clear_collection_data(self, scope_name: str, collection_name: str):
        """Clear all data from a collection."""
        try:
            logger.info(f"Clearing data from {self.bucket_name}.{scope_name}.{collection_name}...")

            # Use N1QL to delete all documents with explicit execution
            delete_query = f"DELETE FROM `{self.bucket_name}`.`{scope_name}`.`{collection_name}`"
            result = self.cluster.query(delete_query)

            # Execute the query and get the results
            rows = list(result)

            # Wait a moment for the deletion to propagate
            time.sleep(2)

            # Verify collection is empty
            count_query = f"SELECT COUNT(*) as count FROM `{self.bucket_name}`.`{scope_name}`.`{collection_name}`"
            count_result = self.cluster.query(count_query)
            count_row = list(count_result)[0]
            remaining_count = count_row["count"]

            if remaining_count == 0:
                logger.info(f"Collection cleared successfully, {remaining_count} documents remaining")
            else:
                logger.warning(f"Collection clear incomplete, {remaining_count} documents remaining")

        except Exception as e:
            logger.warning(f"Error clearing collection data: {e}")
            # If N1QL fails, try to continue anyway
            pass

    def get_collection(self, scope_name: str, collection_name: str):
        """Get a collection object."""
        key = f"{scope_name}.{collection_name}"
        if key not in self._collections:
            self._collections[key] = self.bucket.scope(scope_name).collection(collection_name)
        return self._collections[key]

    def setup_vector_search_index(self, index_definition: dict, scope_name: str):
        """Setup vector search index for the specified scope."""
        try:
            if not self.bucket:
                raise RuntimeError("Bucket not initialized. Call setup_collection first.")

            scope_index_manager = self.bucket.scope(scope_name).search_indexes()
            existing_indexes = scope_index_manager.get_all_indexes()
            index_name = index_definition["name"]

            if index_name not in [index.name for index in existing_indexes]:
                logger.info(f"Creating vector search index '{index_name}'...")
                search_index = SearchIndex.from_json(index_definition)
                scope_index_manager.upsert_index(search_index)
                logger.info(f"Vector search index '{index_name}' created successfully")
            else:
                logger.info(f"Vector search index '{index_name}' already exists")
        except Exception as e:
            raise RuntimeError(f"Error setting up vector search index: {e!s}")

    def setup_vector_store_langchain(self, scope_name, collection_name, index_name, embeddings, data_loader_func):
        """Setup vector store with hotel data using LangChain."""
        try:
            # Load hotel data using the data loading function
            data_loader_func(
                cluster=self.cluster,
                bucket_name=self.bucket_name,
                scope_name=scope_name,
                collection_name=collection_name,
                embeddings=embeddings,
                index_name=index_name,
            )
            logger.info("Hotel data loaded into vector store successfully")

        except Exception as e:
            raise RuntimeError(f"Error setting up vector store: {e!s}")

    def create_langchain_agent(self, catalog, span):
        """Create LangChain ReAct agent with hotel search tool from Agent Catalog."""
        try:
            # Setup AI services using Priority 1: Capella AI + OpenAI wrappers
            embeddings, llm = setup_ai_services(framework="langchain", temperature=0.1, application_span=span)
            
            # Setup collection
            self.setup_collection(os.environ["CB_SCOPE"], os.environ["CB_COLLECTION"], clear_existing_data=False)
            
            # Setup vector search index - MUST have agentcatalog_index.json
            with open("agentcatalog_index.json") as file:
                index_definition = json.load(file)
            logger.info("Loaded vector search index definition from agentcatalog_index.json")
            self.setup_vector_search_index(index_definition, os.environ["CB_SCOPE"])
            
            # Setup vector store with hotel data
            self.setup_vector_store_langchain(
                os.environ["CB_SCOPE"],
                os.environ["CB_COLLECTION"],
                os.environ["CB_INDEX"],
                embeddings,
                load_hotel_data_to_couchbase,
            )
            
            # Load tools and create agent
            tool_search = catalog.find("tool", name="search_vector_database")
            if not tool_search:
                raise ValueError(
                    "Could not find search_vector_database tool. Make sure it's indexed with 'agentc index tools/'"
                )

            tools = [
                Tool(
                    name=tool_search.meta.name,
                    description=tool_search.meta.description,
                    func=tool_search.func,
                ),
            ]

            hotel_prompt = catalog.find("prompt", name="hotel_search_assistant")
            if not hotel_prompt:
                raise ValueError(
                    "Could not find hotel_search_assistant prompt in catalog. Make sure it's indexed with 'agentc index prompts/'"
                )

            custom_prompt = PromptTemplate(
                template=hotel_prompt.content.strip(),
                input_variables=["input", "agent_scratchpad"],
                partial_variables={
                    "tools": "\n".join(
                        [f"{tool.name}: {tool.description}" for tool in tools]
                    ),
                    "tool_names": ", ".join([tool.name for tool in tools]),
                },
            )

            def handle_parsing_error(error) -> str:
                """Custom error handler for parsing errors that guides agent back to ReAct format."""
                logger.warning(f"Parsing error occurred: {error}")
                return """I need to use the correct format. Let me start over:

Thought: I need to search for hotels using the search_vector_database tool
Action: search_vector_database
Action Input: """

            agent = create_react_agent(llm, tools, custom_prompt)
            agent_executor = AgentExecutor(
                agent=agent,
                tools=tools,
                verbose=True,
                handle_parsing_errors=handle_parsing_error,  # Use custom error handler
                max_iterations=2,  # STRICT: 1 tool call + 1 Final Answer only
                max_execution_time=120,  # Force stop when max iterations reached
                early_stopping_method="force",  # Force stop when max iterations reached
                return_intermediate_steps=True,  # For better debugging
            )

            logger.info("LangChain ReAct agent created successfully")
            return agent_executor

        except Exception as e:
            raise RuntimeError(f"Error creating LangChain agent: {e!s}")


## Data Loading Module

Complete hotel data loading functions from data/hotel_data.py - inline for self-contained operation.


In [4]:
# Data loading functions from data/hotel_data.py
import couchbase.auth
import couchbase.cluster
import couchbase.exceptions
import couchbase.options


def retry_with_backoff(func, retries=3):
    """Simple retry with exponential backoff."""
    for attempt in range(retries):
        try:
            return func()
        except Exception:
            if attempt == retries - 1:
                raise
            delay = 2 ** attempt
            logger.warning(f"Attempt {attempt + 1} failed, retrying in {delay}s...")
            time.sleep(delay)


def get_cluster_connection():
    """Get a fresh cluster connection for each request."""
    try:
        auth = couchbase.auth.PasswordAuthenticator(
            username=os.getenv("CB_USERNAME", "Administrator"),
            password=os.getenv("CB_PASSWORD", "password"),
        )
        options = couchbase.options.ClusterOptions(authenticator=auth)
        # Use WAN profile for better timeout handling with remote clusters
        options.apply_profile("wan_development")

        cluster = couchbase.cluster.Cluster(
            os.getenv("CB_CONN_STRING", "couchbase://localhost"), options
        )
        cluster.wait_until_ready(timedelta(seconds=15))
        return cluster
    except couchbase.exceptions.CouchbaseException as e:
        logger.error(f"Could not connect to Couchbase cluster: {str(e)}")
        return None


def load_hotel_data_from_travel_sample():
    """Load hotel data from travel-sample.inventory.hotel collection."""
    try:
        cluster = get_cluster_connection()
        if not cluster:
            raise ConnectionError("Could not connect to Couchbase cluster")

        # Query to get all hotel documents from travel-sample.inventory.hotel
        query = """
            SELECT h.*, META(h).id as doc_id
            FROM `travel-sample`.inventory.hotel h
            ORDER BY h.name
        """

        logger.info("Loading hotel data from travel-sample.inventory.hotel...")
        result = cluster.query(query)

        hotels = []
        for row in result:
            hotel = row
            hotels.append(hotel)

        logger.info(f"Loaded {len(hotels)} hotels from travel-sample.inventory.hotel")
        return hotels

    except Exception as e:
        logger.error(f"Error loading hotel data: {str(e)}")
        raise


def get_hotel_texts():
    """Returns formatted hotel texts for vector store embedding from travel-sample data."""
    hotels = load_hotel_data_from_travel_sample()
    hotel_texts = []

    for hotel in tqdm(hotels, desc="Processing hotels"):
        # Start with basic info
        name = hotel.get("name", "Unknown Hotel")
        city = hotel.get("city", "Unknown City")
        country = hotel.get("country", "Unknown Country")

        # Build text with PRIORITIZED information for search
        text_parts = [f"{name} in {city}, {country}"]

        # PRIORITY 1: Location details (critical for search)
        location_fields = ["address", "state", "directions"]
        for field in location_fields:
            value = hotel.get(field)
            if value and value != "None":
                text_parts.append(f"{field.title()}: {value}")

        # PRIORITY 2: Key amenities (most searched features)
        amenity_fields = [
            ("free_breakfast", "Free breakfast"),
            ("free_internet", "Free internet"), 
            ("free_parking", "Free parking"),
            ("pets_ok", "Pets allowed")
        ]
        for field, label in amenity_fields:
            value = hotel.get(field)
            if value is not None:
                text_parts.append(f"{label}: {'Yes' if value else 'No'}")

        # PRIORITY 3: Hotel description and type
        description_fields = [
            ("description", "Description"),
            ("type", "Type"),
            ("title", "Title")
        ]
        for field, label in description_fields:
            value = hotel.get(field)
            if value and value != "None":
                text_parts.append(f"{label}: {value}")

        # PRIORITY 4: Other details (less critical for search)
        other_fields = [
            ("price", "Price"),
            ("checkin", "Check-in"),
            ("checkout", "Check-out"),
            ("phone", "Phone"),
            ("email", "Email"),
            ("vacancy", "Vacancy"),
            ("alias", "Also known as")
        ]
        for field, label in other_fields:
            value = hotel.get(field)
            if value and value != "None":
                if isinstance(value, bool):
                    text_parts.append(f"{label}: {'Yes' if value else 'No'}")
                else:
                    text_parts.append(f"{label}: {value}")

        # Add geographic coordinates if available
        if hotel.get("geo"):
            geo = hotel["geo"]
            if geo.get("lat") and geo.get("lon"):
                text_parts.append(f"Coordinates: {geo['lat']}, {geo['lon']}")

        # Add review summary if available
        if hotel.get("reviews") and isinstance(hotel["reviews"], list):
            review_count = len(hotel["reviews"])
            if review_count > 0:
                text_parts.append(f"Reviews: {review_count} customer reviews available")

                # Include a sample of review content for better search matching
                sample_reviews = hotel["reviews"][:2]  # First 2 reviews
                for i, review in enumerate(sample_reviews):
                    if review.get("content"):
                        # Truncate long reviews for embedding efficiency
                        content = (
                            review["content"][:200] + "..."
                            if len(review["content"]) > 200
                            else review["content"]
                        )
                        text_parts.append(f"Review {i + 1}: {content}")

        # Add public likes if available
        if hotel.get("public_likes") and isinstance(hotel["public_likes"], list):
            likes_count = len(hotel["public_likes"])
            if likes_count > 0:
                text_parts.append(f"Public likes: {likes_count} likes")

        # Join all parts with ". "
        text = ". ".join(text_parts)
        hotel_texts.append(text)

    logger.info(f"Generated {len(hotel_texts)} hotel text embeddings")
    return hotel_texts


def load_hotel_data_to_couchbase(
    cluster,
    bucket_name: str,
    scope_name: str,
    collection_name: str,
    embeddings,
    index_name: str,
):
    """Load hotel data from travel-sample into the target collection with embeddings."""
    try:
        # Check if data already exists
        count_query = f"SELECT COUNT(*) as count FROM `{bucket_name}`.`{scope_name}`.`{collection_name}`"
        count_result = cluster.query(count_query)
        count_row = list(count_result)[0]
        existing_count = count_row["count"]

        if existing_count > 0:
            logger.info(
                f"Found {existing_count} existing documents in collection, skipping data load"
            )
            return

        # Get hotel texts for embeddings
        hotel_texts = get_hotel_texts()

        # Setup vector store for the target collection
        vector_store = CouchbaseVectorStore(
            cluster=cluster,
            bucket_name=bucket_name,
            scope_name=scope_name,
            collection_name=collection_name,
            embedding=embeddings,
            index_name=index_name,
        )

        # Add hotel texts to vector store with batch processing
        logger.info(
            f"Loading {len(hotel_texts)} hotel embeddings to {bucket_name}.{scope_name}.{collection_name}"
        )

        # Process in batches with simple retry
        batch_size = 10

        with tqdm(total=len(hotel_texts), desc="Loading hotel embeddings") as pbar:
            for i in range(0, len(hotel_texts), batch_size):
                batch = hotel_texts[i : i + batch_size]
                
                def add_batch():
                    return vector_store.add_texts(texts=batch, batch_size=batch_size)
                
                retry_with_backoff(add_batch, retries=3)
                pbar.update(len(batch))

        logger.info(
            f"Successfully loaded {len(hotel_texts)} hotel embeddings to vector store"
        )

    except Exception as e:
        logger.error(f"Error loading hotel data to Couchbase: {str(e)}")
        raise


def get_hotel_count():
    """Get the count of hotels in travel-sample.inventory.hotel."""
    try:
        cluster = get_cluster_connection()
        if not cluster:
            raise ConnectionError("Could not connect to Couchbase cluster")

        query = "SELECT COUNT(*) as count FROM `travel-sample`.inventory.hotel"
        result = cluster.query(query)

        for row in result:
            return row["count"]

        return 0

    except Exception as e:
        logger.error(f"Error getting hotel count: {str(e)}")
        return 0


## Query Module

Complete query collections and functions from data/queries.py - inline for self-contained operation.


In [5]:
# Query functions and data from data/queries.py

# Hotel search queries (based on travel-sample data)
HOTEL_SEARCH_QUERIES = [
    "Find hotels in Giverny with free breakfast",
    "I need a hotel in Glossop with free internet access",
    "Show me hotels in Helensburgh with free breakfast",
]

# Comprehensive reference answers matching actual database content
HOTEL_REFERENCE_ANSWERS = [
    # Query 1: Giverny with free breakfast
    """I found one hotel in Giverny that offers free breakfast:

**Le Clos Fleuri**
- **Location:** Giverny, France  
- **Address:** 5 rue de la Dîme, 27620 Giverny
- **Phone:** +33 2 32 21 36 51
- **Website:** http://www.giverny-leclosfleuri.fr/
- **Amenities:** Free breakfast ✅, Free internet ✅, Free parking ✅, No pets allowed
- **Vacancy:** Yes
- **Coordinates:** 49.0763077, 1.5234464
- **Reviews:** 3 customer reviews available with mixed ratings
- **Public Likes:** 7 likes
- **Description:** Situated near the church and just a few minutes walking distance from Monet's gardens and the Museum of Impressionisms, you will find Danielle and Claude's home, surrounded by a large magnificent garden, where you will find a haven of peace and tranquillity. Danielle speaks fluent English having spent many years in Australia.

This hotel is perfect for your stay in Giverny with the requested free breakfast amenity. It's ideally located for visiting Monet's gardens and offers a peaceful garden setting.""",
    # Query 2: Glossop with free internet
    """Here are hotels in Glossop that offer free internet access:

1. **The George Hotel**
   - **Address:** Norfolk Street, Glossop, United Kingdom
   - **Phone:** +44 1457 855449
   - **Price:** From £35.00 (single) or £60.00 (double)
   - **Amenities:** Free internet ✅, Free breakfast ✅, Pets allowed ✅
   - **Vacancy:** Yes
   - **Reviews:** 6 customer reviews available
   - **Coordinates:** 53.444331, -1.948299
   - **Description:** Set in the centre of town, this hotel makes an ideal base for a visit to the area.

2. **Avondale Guest House**
   - **Address:** 28 Woodhead Road, Glossop, United Kingdom
   - **Phone:** +44 1457 853132, Mobile: +44 7784 764969
   - **Website:** http://www.avondale-guesthouse.co.uk/
   - **Amenities:** Free internet ✅, Free breakfast ✅, Pets allowed ✅
   - **Vacancy:** Yes
   - **Reviews:** 7 customer reviews available
   - **Coordinates:** 53.449979, -1.945284

These hotels are located in Glossop and offer the free internet access you're looking for.""",
    # Query 3: Helensburgh with free breakfast
    """Here are the hotels in Helensburgh that offer free breakfast:

1. **County Lodge Hotel**
   - **Location:** Helensburgh, United Kingdom
   - **Address:** Old Luss Road, Helensburgh, G84 7BH
   - **Phone:** +44 1436 672034
   - **Website:** http://www.countylodgehotel.co.uk/
   - **Amenities:** Free breakfast ✅, Free internet ✅, Free parking ✅, No pets allowed
   - **Price:** Rooms £40-£55
   - **Vacancy:** No
   - **Coordinates:** 55.99884, -4.71354
   - **Description:** Nearly 1 mile east of the town centre, near Colgrain Station.

2. **Commodore Hotel**
   - **Location:** Helensburgh, United Kingdom
   - **Address:** 112-117 West Clyde Street, Helensburgh, G84 8ES
   - **Phone:** +44 1436 676924
   - **Website:** http://www.innkeeperslodge.com/lodgedetail.asp?lid=91
   - **Amenities:** Free breakfast ✅, Free internet ✅, Pets allowed ✅, No free parking
   - **Price:** Rooms from £55
   - **Vacancy:** No
   - **Reviews:** 2 customer reviews available
   - **Coordinates:** 56.00481, -4.74472
   - **Description:** The biggest hotel in town with rooms from £55. Refurbished in about 2004. On the sea front about 1/2 mile from the town centre.

Both hotels offer the requested free breakfast along with additional amenities.""",
]

# Create dictionary for backward compatibility
QUERY_REFERENCE_ANSWERS = {
    query: answer
    for query, answer in zip(HOTEL_SEARCH_QUERIES, HOTEL_REFERENCE_ANSWERS)
}


def get_evaluation_queries():
    """Get queries for evaluation"""
    return HOTEL_SEARCH_QUERIES


def get_reference_answer(query: str) -> str:
    """Get the correct reference answer for a given query"""
    return QUERY_REFERENCE_ANSWERS.get(
        query, f"No reference answer available for: {query}"
    )


## Hotel Search Agent Setup

Setup the complete hotel search agent infrastructure using LangChain.


In [6]:
def setup_hotel_support_agent():
    """Setup the hotel support agent with Agent Catalog integration."""
    try:
        # Initialize Agent Catalog with single application span
        catalog = agentc.catalog.Catalog()
        application_span = catalog.Span(name="Hotel Support Agent", blacklist=set())

        # Setup environment
        setup_environment()

        # Test Capella AI connectivity if configured
        if os.getenv("CAPELLA_API_ENDPOINT"):
            if not test_capella_connectivity():
                logger.warning(
                    "❌ Capella AI connectivity test failed. Will use OpenAI fallback."
                )
        else:
            logger.info("ℹ️ Capella API not configured - will use OpenAI models")

        # Setup Couchbase connection and collections using CouchbaseClient
        couchbase_client = CouchbaseClient(
            conn_string=os.getenv("CB_CONN_STRING", "couchbase://localhost"),
            username=os.getenv("CB_USERNAME", "Administrator"),
            password=os.getenv("CB_PASSWORD", "password"),
            bucket_name=os.getenv("CB_BUCKET", DEFAULT_BUCKET),
        )
        couchbase_client.connect()

        # Create agent using the CouchbaseClient
        agent_executor = couchbase_client.create_langchain_agent(catalog, application_span)

        return agent_executor, application_span

    except Exception as e:
        logger.exception(f"Error setting up hotel support agent: {e}")
        raise


# Setup the hotel search agent
agent, span = setup_hotel_support_agent()


2025-09-11 13:31:50,621 - agentc_core.catalog.catalog - INFO - A local catalog and a remote catalog have been found. Building a chained tool catalog.
2025-09-11 13:31:50,621 - agentc_core.catalog.catalog - INFO - A local catalog and a remote catalog have been found. Building a chained prompt catalog.
2025-09-11 13:31:50,675 - agentc_core.activity.span - INFO - Using both a local auditor and a remote auditor.
2025-09-11 13:31:50,675 - __main__ - INFO - ✅ Environment variables configured
2025-09-11 13:32:08,418 - __main__ - INFO - Successfully connected to Couchbase
2025-09-11 13:32:08,419 - __main__ - INFO - 🔧 Setting up Priority 1 AI services for langchain framework...
2025-09-11 13:32:08,419 - __main__ - INFO - 🔧 Endpoint: https://hm6yblbdkwamqjlq.ai.sandbox.nonprod-project-avengers.com
2025-09-11 13:32:08,420 - __main__ - INFO - 🔧 Model: nvidia/llama-3.2-nv-embedqa-1b-v2
2025-09-11 13:32:08,420 - __main__ - INFO - 🔧 API Base: https://hm6yblbdkwamqjlq.ai.sandbox.nonprod-project-avenge

## Test Functions
Define test functions to demonstrate the hotel search agent functionality.


In [7]:
def run_hotel_query(query: str, agent):
    """Run a single hotel query with error handling."""
    logger.info(f"🏨 Hotel Query: {query}")
    
    try:
        # Run the agent with LangChain invoke interface
        response = agent.invoke({"input": query})
        result = response["output"]
        
        logger.info(f"🤖 AI Response: {result}")
        logger.info("✅ Query completed successfully")
        
        return result
        
    except Exception as e:
        logger.exception(f"❌ Query failed: {e}")
        return f"Error: {str(e)}"


def test_hotel_data_loading():
    """Test hotel data loading from travel-sample independently."""
    logger.info("Testing Hotel Data Loading from travel-sample")
    logger.info("=" * 50)
    
    try:
        # Test hotel count
        count = get_hotel_count()
        logger.info(f"✅ Hotel count in travel-sample.inventory.hotel: {count}")
        
        # Test hotel text generation
        texts = get_hotel_texts()
        logger.info(f"✅ Generated {len(texts)} hotel texts for embeddings")
        
        if texts:
            logger.info(f"✅ First hotel text sample: {texts[0][:200]}...")
        
        logger.info("✅ Data loading test completed successfully")
        
    except Exception as e:
        logger.exception(f"❌ Data loading test failed: {e}")


# Test hotel data loading
test_hotel_data_loading()


2025-09-11 13:32:21,021 - __main__ - INFO - Testing Hotel Data Loading from travel-sample
2025-09-11 13:32:21,021 - __main__ - INFO - ==================================================
2025-09-11 13:32:23,929 - __main__ - INFO - ✅ Hotel count in travel-sample.inventory.hotel: 917
2025-09-11 13:32:26,336 - __main__ - INFO - Loading hotel data from travel-sample.inventory.hotel...
2025-09-11 13:32:29,298 - __main__ - INFO - Loaded 917 hotels from travel-sample.inventory.hotel


Processing hotels: 100%|██████████| 917/917 [00:00<00:00, 155715.66it/s]

2025-09-11 13:32:29,323 - __main__ - INFO - Generated 917 hotel text embeddings
2025-09-11 13:32:29,325 - __main__ - INFO - ✅ Generated 917 hotel texts for embeddings
2025-09-11 13:32:29,326 - __main__ - INFO - ✅ First hotel text sample: 'La Mirande Hotel in Avignon, France. Address: 4 place de la Mirande,F- AVIGNON. State: Provence-Alpes-Côte d'Azur. Free breakfast: Yes. Free internet: Yes. Free parking: No. Pets allowed: Yes. Descri...
2025-09-11 13:32:29,326 - __main__ - INFO - ✅ Data loading test completed successfully


## Test 1: Hotels in Giverny with Free Breakfast

Search for hotels in Giverny, France that offer free breakfast.


In [8]:
result1 = run_hotel_query("Find hotels in Giverny with free breakfast", agent)


2025-09-11 13:32:29,341 - __main__ - INFO - 🏨 Hotel Query: Find hotels in Giverny with free breakfast


> Entering new AgentExecutor chain...
Thought: I should search for hotels matching the user's request 
Action: search_vector_database 
Action Input: "hotels in Giverny with free breakfast"
Observation

/Users/kaustavghosh/Desktop/agent-catalog-quickstart/notebooks/hotel_search_agent_langchain/tools/search_vector_database.py:102: LangChainDeprecationWarning: The class `CouchbaseVectorStore` was deprecated in LangChain 0.3.0 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-couchbase package and should be used instead. To use it run `pip install -U :class:`~langchain-couchbase` and import as `from :class:`~langchain_couchbase.vectorstores import CouchbaseSearchVectorStore``.
  vector_store = CouchbaseVectorStore(


FOUND_6_HOTELS:
HOTEL_1: Le Clos Fleuri in Giverny, France. Address: 5 rue de la Dîme. State: Haute-Normandie. Directions: 27620 Giverny. Free breakfast: Yes. Free internet: Yes. Free parking: Yes. Pets allowed: No. Description: Situated near the church and just a few minutes walking distance from Monet's gardens and the Museum of Impressionisms, you will find Danielle and Claude's home, surrounded by a large magnificent garden, where you will find a haven of peace and tranquillity. Danielle speaks fluent English having spent many years in Australia.. Type: hotel. Title: Giverny. Phone: +33 2 32 21 36 51. Vacancy: Yes. Coordinates: 49.0763077, 1.5234464. Reviews: 3 customer reviews available. Review 1: Very basic place to stay with adjoining buildings still run down from Katrina. If you have a car and looking for good value this is perfect. Complimentary Breakfast is adequate for what you pay. Overa.... Review 2: the bed were never cleaned, the same linens were on the bed. i would like

## Test 2: Hotels in Glossop with Free Internet

Search for hotels in Glossop, UK that offer free internet access.


In [9]:
result2 = run_hotel_query("I need a hotel in Glossop with free internet access", agent)


2025-09-11 13:32:58,916 - __main__ - INFO - 🏨 Hotel Query: I need a hotel in Glossop with free internet access


> Entering new AgentExecutor chain...
Thought: I should search for hotels matching the user's request 
Action: search_vector_database 
Action Input: "hotel in Glossop with free internet access"
ObservationFOUND_6_HOTELS:
HOTEL_1: Avondale Guest House in Glossop, United Kingdom. Address: 28 Woodhead Road. Free breakfast: Yes. Free internet: Yes. Free parking: No. Pets allowed: Yes. Description: Mobile: +44 7784 764969. Type: hotel. Title: Glossop. Phone: +44 1457 853132. Vacancy: Yes. Coordinates: 53.449979, -1.945284. Reviews: 7 customer reviews available. Review 1: I tagged along on my husband's work trip (not expensed) and had a great time. I think we made the perfect choice. PROS: 1. Location is fantastic. If you head out the back entrance you are a block from.... Review 2: I lived in New Orleans while completing my residency at Ochsner, but this was prior to The Ritz com

## Test 3: Hotels in Helensburgh with Free Breakfast

Search for hotels in Helensburgh, Scotland that offer free breakfast.


In [10]:
result3 = run_hotel_query("Show me hotels in Helensburgh with free breakfast", agent)


2025-09-11 13:33:12,283 - __main__ - INFO - 🏨 Hotel Query: Show me hotels in Helensburgh with free breakfast


> Entering new AgentExecutor chain...
Thought: I should search for hotels matching the user's request 
Action: search_vector_database 
Action Input: "Helensburgh hotels with free breakfast"
ObservationFOUND_6_HOTELS:
HOTEL_1: Imperial Hotel in Helensburgh, United Kingdom. Address: 12-14 West Clyde St,Helensburgh, G84 8SQ.. Free breakfast: No. Free internet: Yes. Free parking: No. Pets allowed: No. Description: In the centre of town on the sea front.. Type: hotel. Title: Helensburgh. Phone: +44 1436 672320. Coordinates: 56.00308, -4.73468. Reviews: 2 customer reviews available. Review 1: we stated at this hotel for only one nite and wished we had stayed here on a previous trip. The staff at check in are fantastic, the hotel is modern, clean, comfortable, and has a fantastic free inter.... Review 2: Ibis on Bencoolen street would steal your heart (and not your money) and leave m

## Comprehensive Phoenix Evaluation System

Complete Phoenix evaluation system from evals/eval_arize.py - inline for self-contained operation.


In [11]:
# Phoenix evaluation dependencies and configuration
import json
import socket
import subprocess
import sys
import warnings
import time
import os
import logging
from datetime import datetime
from typing import Dict, List, Optional, Tuple, Any
from dataclasses import dataclass
import pandas as pd
import nest_asyncio

# Apply nest_asyncio to handle nested event loops
nest_asyncio.apply()

# Suppress warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sqlalchemy")
warnings.filterwarnings("ignore", message=".*expression-based index.*")

# Try to import Phoenix dependencies
try:
    import phoenix as px
    from openinference.instrumentation.langchain import LangChainInstrumentor
    from openinference.instrumentation.openai import OpenAIInstrumentor
    from phoenix.evals import (
        HALLUCINATION_PROMPT_RAILS_MAP,
        HALLUCINATION_PROMPT_TEMPLATE,
        QA_PROMPT_RAILS_MAP,
        QA_PROMPT_TEMPLATE,
        RAG_RELEVANCY_PROMPT_RAILS_MAP,
        RAG_RELEVANCY_PROMPT_TEMPLATE,
        TOXICITY_PROMPT_RAILS_MAP,
        TOXICITY_PROMPT_TEMPLATE,
        HallucinationEvaluator,
        OpenAIModel,
        QAEvaluator,
        RelevanceEvaluator,
        ToxicityEvaluator,
        llm_classify,
        run_evals,
    )
    from phoenix.otel import register
    ARIZE_AVAILABLE = True
    logger.info("✅ Phoenix dependencies available")
except ImportError as e:
    logger.warning(f"Phoenix dependencies not available: {e}")
    ARIZE_AVAILABLE = False

@dataclass
class EvaluationConfig:
    """Configuration for the evaluation system."""
    project_name: str = "hotel-search-agent-evaluation"
    phoenix_base_port: int = 6007
    evaluator_model: str = "gpt-4o"
    max_queries: int = 10

class PhoenixManager:
    """Manages Phoenix server lifecycle."""
    
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.session = None
        self.active_port = None
        self.tracer_provider = None

    def _kill_existing_phoenix_processes(self) -> None:
        """Kill any existing Phoenix processes."""
        try:
            subprocess.run(["pkill", "-f", "phoenix"], check=False, capture_output=True)
            time.sleep(2)  # Wait for processes to terminate
        except Exception as e:
            logger.debug(f"Error killing Phoenix processes: {e}")

    def start_phoenix(self) -> bool:
        if not ARIZE_AVAILABLE:
            logger.warning("⚠️ Phoenix dependencies not available")
            return False
        try:
            # Kill existing Phoenix processes first
            self._kill_existing_phoenix_processes()
            logger.info("🔧 Setting up Phoenix observability...")
            self.session = px.launch_app(port=self.config.phoenix_base_port)
            self.active_port = self.config.phoenix_base_port
            if self.session:
                logger.info(f"🌐 Phoenix UI: {self.session.url}")
            self.tracer_provider = register(
                project_name=self.config.project_name,
                endpoint=f"http://localhost:{self.config.phoenix_base_port}/v1/traces",
            )
            logger.info("✅ Phoenix setup completed successfully")
            return True
        except Exception as e:
            logger.exception(f"❌ Phoenix setup failed: {e}")
            return False

    def setup_instrumentation(self) -> bool:
        if not self.tracer_provider or not ARIZE_AVAILABLE:
            return False
        try:
            instrumentors = [("LangChain", LangChainInstrumentor), ("OpenAI", OpenAIInstrumentor)]
            for name, instrumentor_class in instrumentors:
                try:
                    instrumentor = instrumentor_class()
                    instrumentor.instrument(tracer_provider=self.tracer_provider)
                    logger.info(f"✅ {name} instrumentation enabled")
                except Exception as e:
                    logger.warning(f"⚠️ {name} instrumentation failed: {e}")
            return True
        except Exception as e:
            logger.exception(f"❌ Instrumentation setup failed: {e}")
            return False


2025-09-11 13:33:37,781 - phoenix.config - INFO - 📋 Ensuring phoenix working directory: /Users/kaustavghosh/.phoenix
2025-09-11 13:33:37,790 - phoenix.inferences.inferences - INFO - Dataset: phoenix_inferences_a3c809d5-940e-41bb-8581-c58f54a9fd3f initialized
2025-09-11 13:33:40,124 - __main__ - INFO - ✅ Phoenix dependencies available


## Arize Phoenix Evaluation

This section demonstrates how to evaluate the hotel search agent using Arize Phoenix observability platform.


In [12]:
# Phoenix evaluation demo
if ARIZE_AVAILABLE:
    try:
        # Start Phoenix
        config = EvaluationConfig(phoenix_base_port=6007)
        phoenix_manager = PhoenixManager(config)
        
        if phoenix_manager.start_phoenix():
            phoenix_manager.setup_instrumentation()
            
            # Run demo queries
            demo_queries = get_evaluation_queries()
            demo_results = []
            
            for i, query in enumerate(demo_queries, 1):
                try:
                    logger.info(f"🔍 Query {i}: {query}")
                    response = agent.invoke({"input": query})
                    output = response["output"]
                    demo_results.append({
                        "query": query,
                        "response": output,
                        "success": True
                    })
                except Exception as e:
                    demo_results.append({
                        "query": query,
                        "response": f"Error: {e}",
                        "success": False
                    })
            
            # Convert to DataFrame for evaluation
            hotel_results_df = pd.DataFrame(demo_results)
            logger.info(f"📊 Collected {len(hotel_results_df)} responses for evaluation")
            
            logger.info(f"🚀 Phoenix UI: http://localhost:{config.phoenix_base_port}/")
            logger.info("💡 Visit Phoenix UI for detailed traces")
            
    except Exception as e:
        logger.exception(f"Phoenix evaluation failed: {e}")
        
else:
    logger.info("Phoenix not available - install phoenix-evals")


2025-09-11 13:33:42,184 - __main__ - INFO - 🔧 Setting up Phoenix observability...
2025-09-11 13:33:42,189 - phoenix.config - INFO - 📋 Ensuring phoenix working directory: /Users/kaustavghosh/.phoenix
❗️ The launch_app `port` parameter is deprecated and will be removed in a future release. Use the `PHOENIX_PORT` environment variable instead.
2025-09-11 13:33:42,282 - alembic.runtime.migration - INFO - Context impl SQLiteImpl.
2025-09-11 13:33:42,283 - alembic.runtime.migration - INFO - Will assume transactional DDL.
2025-09-11 13:33:42,301 - alembic.runtime.migration - INFO - Running upgrade  -> cf03bd6bae1d, init
2025-09-11 13:33:42,337 - alembic.runtime.migration - INFO - Running upgrade cf03bd6bae1d -> 10460e46d750, datasets
2025-09-11 13:33:42,344 - alembic.runtime.migration - INFO - Running upgrade 10460e46d750 -> 3be8647b87d8, add token columns to spans table
2025-09-11 13:33:42,346 - alembic.runtime.migration - INFO - Running upgrade 3be8647b87d8 -> cd164e83824f, users and tokens


In [13]:
# Run comprehensive Phoenix evaluations with lenient templates
if ARIZE_AVAILABLE and len(demo_results) > 0:
    logger.info("🔍 Running comprehensive Phoenix evaluations...")

    # Setup evaluator LLM
    evaluator_llm = OpenAIModel(model="gpt-4o", temperature=0.1)

    # Prepare evaluation data
    hotel_eval_data = []
    for _, row in hotel_results_df.iterrows():
        hotel_eval_data.append({
            "input": row["query"],
            "output": row["response"],
            "reference": get_reference_answer(row["query"]),
            "text": row["response"],  # For toxicity evaluation
        })

    hotel_eval_df = pd.DataFrame(hotel_eval_data)
    logger.info(f"📊 Prepared {len(hotel_eval_df)} queries for Phoenix evaluation")

    try:
        # 1. Relevance Evaluation
        logger.info("🔍 Running Relevance Evaluation...")
        hotel_relevance_results = llm_classify(
            data=hotel_eval_df[["input", "reference"]],
            model=evaluator_llm,
            template=RAG_RELEVANCY_PROMPT_TEMPLATE,
            rails=list(RAG_RELEVANCY_PROMPT_RAILS_MAP.values()),
            provide_explanation=True,
        )
        
        logger.info("✅ Relevance Evaluation Results:")
        relevance_labels = hotel_relevance_results['label'].tolist() if 'label' in hotel_relevance_results.columns else []
        for i, (query, label) in enumerate(zip(hotel_eval_df['input'], relevance_labels)):
            logger.info(f"   Query: {query}")
            logger.info(f"   Relevance: {label}")
            logger.info("   " + "-"*30)
        
        # 2. QA Evaluation
        logger.info("🔍 Running QA Evaluation...")
        hotel_qa_results = llm_classify(
            data=hotel_eval_df[["input", "output", "reference"]],
            model=evaluator_llm,
            template=QA_PROMPT_TEMPLATE,
            rails=list(QA_PROMPT_RAILS_MAP.values()),
            provide_explanation=True,
        )
        
        logger.info("✅ QA Evaluation Results:")
        qa_labels = hotel_qa_results['label'].tolist() if 'label' in hotel_qa_results.columns else []
        for i, (query, label) in enumerate(zip(hotel_eval_df['input'], qa_labels)):
            logger.info(f"   Query: {query}")
            logger.info(f"   QA Score: {label}")
            logger.info("   " + "-"*30)
        
        # 3. Hallucination Evaluation
        logger.info("🔍 Running Hallucination Evaluation...")
        hotel_hallucination_results = llm_classify(
            data=hotel_eval_df[["input", "reference", "output"]],
            model=evaluator_llm,
            template=HALLUCINATION_PROMPT_TEMPLATE,
            rails=list(HALLUCINATION_PROMPT_RAILS_MAP.values()),
            provide_explanation=True,
        )
        
        logger.info("✅ Hallucination Evaluation Results:")
        hallucination_labels = hotel_hallucination_results['label'].tolist() if 'label' in hotel_hallucination_results.columns else []
        for i, (query, label) in enumerate(zip(hotel_eval_df['input'], hallucination_labels)):
            logger.info(f"   Query: {query}")
            logger.info(f"   Hallucination: {label}")
            logger.info("   " + "-"*30)
        
        # 4. Toxicity Evaluation
        logger.info("🔍 Running Toxicity Evaluation...")
        hotel_toxicity_results = llm_classify(
            data=hotel_eval_df[["input"]],
            model=evaluator_llm,
            template=TOXICITY_PROMPT_TEMPLATE,
            rails=list(TOXICITY_PROMPT_RAILS_MAP.values()),
            provide_explanation=True,
        )
        
        logger.info("✅ Toxicity Evaluation Results:")
        toxicity_labels = hotel_toxicity_results['label'].tolist() if 'label' in hotel_toxicity_results.columns else []
        for i, (query, label) in enumerate(zip(hotel_eval_df['input'], toxicity_labels)):
            logger.info(f"   Query: {query}")
            logger.info(f"   Toxicity: {label}")
            logger.info("   " + "-"*30)
        
        # Summary
        logger.info("\n📊 Phoenix Evaluation Summary:")
        logger.info(f"  Relevance: {dict(pd.Series(relevance_labels).value_counts())}")
        logger.info(f"  QA Correctness: {dict(pd.Series(qa_labels).value_counts())}")
        logger.info(f"  Hallucination: {dict(pd.Series(hallucination_labels).value_counts())}")
        logger.info(f"  Toxicity: {dict(pd.Series(toxicity_labels).value_counts())}")
        
    except Exception as e:
        logger.exception(f"❌ Phoenix evaluation failed: {e}")

else:
    logger.info("⚠️ Skipping Phoenix evaluations - no demo results available")


2025-09-11 13:34:27,506 - __main__ - INFO - 🔍 Running comprehensive Phoenix evaluations...
2025-09-11 13:34:27,520 - __main__ - INFO - 📊 Prepared 3 queries for Phoenix evaluation
2025-09-11 13:34:27,521 - __main__ - INFO - 🔍 Running Relevance Evaluation...


llm_classify |          | 0/3 (0.0%) | ⏳ 00:00<? | ?it/s

2025-09-11 13:34:36,062 - __main__ - INFO - ✅ Relevance Evaluation Results:
2025-09-11 13:34:36,065 - __main__ - INFO -    Query: Find hotels in Giverny with free breakfast
2025-09-11 13:34:36,066 - __main__ - INFO -    Relevance: relevant
2025-09-11 13:34:36,067 - __main__ - INFO -    ------------------------------
2025-09-11 13:34:36,068 - __main__ - INFO -    Query: I need a hotel in Glossop with free internet access
2025-09-11 13:34:36,068 - __main__ - INFO -    Relevance: relevant
2025-09-11 13:34:36,069 - __main__ - INFO -    ------------------------------
2025-09-11 13:34:36,070 - __main__ - INFO -    Query: Show me hotels in Helensburgh with free breakfast
2025-09-11 13:34:36,070 - __main__ - INFO -    Relevance: relevant
2025-09-11 13:34:36,071 - __main__ - INFO -    ------------------------------
2025-09-11 13:34:36,072 - __main__ - INFO - 🔍 Running QA Evaluation...


llm_classify |          | 0/3 (0.0%) | ⏳ 00:00<? | ?it/s

2025-09-11 13:34:45,168 - __main__ - INFO - ✅ QA Evaluation Results:
2025-09-11 13:34:45,170 - __main__ - INFO -    Query: Find hotels in Giverny with free breakfast
2025-09-11 13:34:45,171 - __main__ - INFO -    QA Score: incorrect
2025-09-11 13:34:45,171 - __main__ - INFO -    ------------------------------
2025-09-11 13:34:45,172 - __main__ - INFO -    Query: I need a hotel in Glossop with free internet access
2025-09-11 13:34:45,173 - __main__ - INFO -    QA Score: incorrect
2025-09-11 13:34:45,174 - __main__ - INFO -    ------------------------------
2025-09-11 13:34:45,175 - __main__ - INFO -    Query: Show me hotels in Helensburgh with free breakfast
2025-09-11 13:34:45,175 - __main__ - INFO -    QA Score: correct
2025-09-11 13:34:45,176 - __main__ - INFO -    ------------------------------
2025-09-11 13:34:45,177 - __main__ - INFO - 🔍 Running Hallucination Evaluation...


llm_classify |          | 0/3 (0.0%) | ⏳ 00:00<? | ?it/s

2025-09-11 13:34:52,792 - __main__ - INFO - ✅ Hallucination Evaluation Results:
2025-09-11 13:34:52,794 - __main__ - INFO -    Query: Find hotels in Giverny with free breakfast
2025-09-11 13:34:52,794 - __main__ - INFO -    Hallucination: hallucinated
2025-09-11 13:34:52,795 - __main__ - INFO -    ------------------------------
2025-09-11 13:34:52,796 - __main__ - INFO -    Query: I need a hotel in Glossop with free internet access
2025-09-11 13:34:52,797 - __main__ - INFO -    Hallucination: hallucinated
2025-09-11 13:34:52,798 - __main__ - INFO -    ------------------------------
2025-09-11 13:34:52,798 - __main__ - INFO -    Query: Show me hotels in Helensburgh with free breakfast
2025-09-11 13:34:52,799 - __main__ - INFO -    Hallucination: hallucinated
2025-09-11 13:34:52,799 - __main__ - INFO -    ------------------------------
2025-09-11 13:34:52,800 - __main__ - INFO - 🔍 Running Toxicity Evaluation...


llm_classify |          | 0/3 (0.0%) | ⏳ 00:00<? | ?it/s

2025-09-11 13:34:59,254 - __main__ - INFO - ✅ Toxicity Evaluation Results:
2025-09-11 13:34:59,255 - __main__ - INFO -    Query: Find hotels in Giverny with free breakfast
2025-09-11 13:34:59,255 - __main__ - INFO -    Toxicity: non-toxic
2025-09-11 13:34:59,255 - __main__ - INFO -    ------------------------------
2025-09-11 13:34:59,255 - __main__ - INFO -    Query: I need a hotel in Glossop with free internet access
2025-09-11 13:34:59,255 - __main__ - INFO -    Toxicity: non-toxic
2025-09-11 13:34:59,256 - __main__ - INFO -    ------------------------------
2025-09-11 13:34:59,256 - __main__ - INFO -    Query: Show me hotels in Helensburgh with free breakfast
2025-09-11 13:34:59,256 - __main__ - INFO -    Toxicity: non-toxic
2025-09-11 13:34:59,256 - __main__ - INFO -    ------------------------------
2025-09-11 13:34:59,256 - __main__ - INFO - 
📊 Phoenix Evaluation Summary:
2025-09-11 13:34:59,257 - __main__ - INFO -   Relevance: {'relevant': np.int64(3)}
2025-09-11 13:34:59,258 -

## Summary

This notebook demonstrates a complete hotel search agent using LangChain, Couchbase vector store, and Capella AI. The agent handles hotel search queries with amenity filtering and location-based recommendations using real travel-sample data.

## Key Features

1. **LangChain ReAct Agent**: Uses LangChain's ReAct pattern for reasoning and action
2. **Couchbase Vector Store**: Real travel-sample hotel data with vector search
3. **Capella AI Integration**: Priority 1 AI services with OpenAI fallback
4. **Agent Catalog**: Tool and prompt discovery and integration
5. **Phoenix Evaluation**: Comprehensive evaluation with relevance, QA, hallucination, and toxicity metrics

## Phoenix Evaluation Metrics

The notebook demonstrates all four key Phoenix evaluation types:

1. **Relevance Evaluation**: Measures how relevant responses are to hotel queries
2. **QA Evaluation**: Assesses the quality and accuracy of hotel information
3. **Hallucination Detection**: Identifies fabricated or incorrect hotel information
4. **Toxicity Detection**: Screens for harmful or inappropriate content
